<a href="https://colab.research.google.com/github/zcvpal2/Aiffel-follow-deep-learning/blob/main/5_aiffel_%EA%B1%B0%EC%9A%B8%EC%95%84_%EA%B1%B0%EC%9A%B8%EC%95%84_(2)_%EC%A0%84%EC%9D%B4_%ED%95%99%EC%8A%B5_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

dataset = 'cats_vs_dogs'
DATA_DIR = '/content/drive/MyDrive/AIFFEL'
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    dataset,
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    data_dir=DATA_DIR,  
    with_info=True,
    download = False,
    as_supervised=True,)

IMG_SIZE = 160 

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
  return image,label  

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example) 

BATCH_SIZE = 32 
SHUFFLE = 1000 

train_batches = train.shuffle(SHUFFLE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE) 
test_batches = test.batch(BATCH_SIZE)

for image_batch, label_batch in train_batches.take(1):
  pass

In [9]:
## VGG 모델 불러오는데 output에 가까운 3개의 FC layer는 제외하기 위해
## include_top =False 해야 

IMG_SHAPE = (IMG_SIZE, IMG_SIZE,3)

base_model = tf.keras.applications.VGG16(input_shape = IMG_SHAPE,
                                         include_top = False,
                                         weights = 'imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [10]:
## base_model에 배치를 넣으면 어떻게 변할지 보면
feature_batch = base_model(image_batch)
feature_batch.shape

TensorShape([32, 5, 5, 512])

In [11]:
#기존의 이미지는
image_batch.shape

TensorShape([32, 160, 160, 3])

In [12]:
# base 모델에 넣은 이미지를 보면 height, width 작아지고, channel은 512로 늘어났다.
# 이렇게 모델에 이미지를 넣어 특징을 추출한 결과를 특징벡터라고 한다.

In [13]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 160, 160, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 160, 160, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 80, 80, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 80, 80, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 80, 80, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 40, 40, 128)       0     

In [14]:
## VGG16 classifier 레이어를 원하는 분류 모델 생성

## 분류모델로 만들기 위해선 Fully connected 레이어로 구성
## 이를 위해서는 반드시 1차원이어야 한다. Flatten 
## 또 다른 방법으로는 Global Average Pooling : 평균값을 활용해 벡터의 차원을 줄이기
# (Height * Width * Depth) >> Global Average Pooling >> (1*1*Depth)

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 512)


In [17]:
## 마지막으로 분류 모델 생성
dense_layer = tf.keras.layers.Dense(512, activation ='relu')
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')

In [18]:
## feature_batch_average가 dense_layer로
# dense_layer가 prediction_layer로

prediction_batch = prediction_layer(dense_layer(feature_batch_average))
print(prediction_batch.shape)

(32, 2)


In [19]:
### 이렇게 원하는 모델을 형성하였고
## vgg16에 해당하는 base_model은 학습시키지 않을것이기 때문에
# trainable을 False로
base_model.trainable = False


In [20]:
model = tf.keras.Sequential([
    base_model, ## vgg16
    global_average_layer, ## flatten 대신 GlobalAveragePooling2D
    dense_layer, 
    prediction_layer ## 고양이인지 개인지 맞추기 위해 이중분류
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 5, 5, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14,978,370
Trainable params: 263,682
Non-trainable params: 14,714,688
_________________________________________________________________
